In [2]:
import numpy as np
import sklearn
from sklearn import svm
from sklearn.neural_network import MLPClassifier as MLPC
from sklearn import preprocessing
import io
import pandas as pd
print(sklearn.__version__)

0.19.1


## Training Data

In [7]:
training = pd.read_csv('training_data.csv')
# Uncoment deletes if reuplod
#df2['tracking'] = pd.Series(np.random.randint(0, 2, 99), index=df2.index)
#training.drop(labels='path', axis=1)
del training['path']
del training['path_specified']
del training['port']
del training['port_specified']

## Collected Data

In [8]:
inp = pd.read_csv('data_collected.csv')
del inp['path']
del inp['path_specified']
del inp['port']
del inp['port_specified']

## Converting string to binary (training data)

In [9]:
training['secure'] = (training['secure'] == True).astype(int)
training['discard'] = (training['discard'] == True).astype(int)
training['expires'] = training['expires'].fillna(0)

## Converting string to int (collected data)

In [10]:
inp['secure'] = (inp['secure'] == True).astype(int)
inp['discard'] = (inp['discard'] == True).astype(int)
inp['expires'] = inp['expires'].fillna(0)

In [11]:
training.dtypes
training['expires'].astype(np.int64)
training.dtypes

Url          object
secure        int64
expires     float64
discard       int64
tracking      int64
dtype: object

In [12]:
inp.dtypes
inp['expires'].astype(np.int64)
inp.dtypes

URL         object
secure       int64
expires    float64
discard      int64
dtype: object

## Inserting data into a matrix

In [13]:
data = training.as_matrix(['secure', 'expires', 'discard', 'tracking'])

In [14]:
inpData = inp.as_matrix(['secure', 'expires', 'discard', 'tracking'])

In [15]:
print(data.shape)
print(data[8, 0:3])
x_train = data[:, 0:3]
labels = data[:, 3]
print(x_train.shape)
print(labels.shape)

(99, 4)
[ 0.  0.  1.]
(99, 3)
(99,)


In [16]:
print(inpData.shape)
print(inpData[8, 0:3])
cookie_data = inpData[:, 0:3]
inpLabels = data[:,3]

print(inpLabels.shape)

(558, 4)
[ 1.  0.  1.]
(99,)


## Pre Processing

In [17]:
##scalling data
scaler = preprocessing.StandardScaler().fit(cookie_data)
input_scaled = preprocessing.scale(cookie_data)
##train
x_train
train_scaler = preprocessing.StandardScaler().fit(x_train)
x_train_scaler = preprocessing.scale(x_train)

##normalising data
normalizer = preprocessing.Normalizer().fit(cookie_data)
input_normalized = normalizer.transform(cookie_data)
##train
train_normalizer = preprocessing.Normalizer().fit(x_train)
x_train_norm = train_normalizer.transform(x_train)

##min max
min_max_scaler = preprocessing.MinMaxScaler()
input_min_max = min_max_scaler.fit_transform(cookie_data)
##train
x_train_min = min_max_scaler.fit_transform(x_train)

/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


# Grid Search

In [18]:
from sklearn.model_selection import GridSearchCV

In [19]:
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [20]:
svc_param_selection(x_train,labels,3)

{'C': 1, 'gamma': 1}

In [21]:
model = svm.SVC(kernel='rbf')

## SVM

In [22]:
model = svm.SVC(kernel='rbf', gamma=1)
model.fit(x_train, labels)
predictions = [model.predict(np.array(cookie_dat).reshape(1, 3))[0] for cookie_dat in cookie_data]
print(predictions)
list = []
for x in predictions:
    if x == 0.0:
        list.append(x)
print("The number of times 0 appears:")
len(list)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

20

## SVM (normalized data)

In [23]:
model = svm.SVC(kernel='rbf', gamma=1)
model.fit(x_train_norm, labels)
predictions = [model.predict(np.array(cookie_dat).reshape(1, 3))[0] for cookie_dat in input_normalized]
print(predictions)
list = []
for x in predictions:
    if x == 1.0:
        list.append(x)
print("The number of time 1 appears:")
len(list)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

538

## SVM (Standardized data)

In [24]:
model = svm.SVC(kernel='rbf', gamma=1)
model.fit(x_train_scaler, labels)
predictions = [model.predict(np.array(cookie_dat).reshape(1, 3))[0] for cookie_dat in input_scaled]
print(predictions)
list = []
for x in predictions:
    if x == 0.0:
        list.append(x)
print("The number of times 0 appears:")
len(list)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

0

## SVM min max

In [25]:
model = svm.SVC(kernel='rbf', gamma=1)
model.fit(x_train_min, labels)
predictions = [model.predict(np.array(cookie_dat).reshape(1, 3))[0] for cookie_dat in input_min_max]
print(predictions)
list = []
for x in predictions:
    if x == 0.0:
        list.append(x)
print("The number of times 0 appears:")
len(list)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

20

## This MLP CLassifier test

In [26]:
model = MLPC([20, 20], activation='relu', solver='lbfgs', learning_rate_init=0.001, max_iter=200, shuffle=True, tol=1e-8, verbose=1)
model.fit(x_train, labels)
pred = [model.predict(np.array(cookie_dat).reshape(1, 3))[0] for cookie_dat in cookie_data]
print("Target {} -----> Predicted {}".format(labels[0:2], pred))
list = []
for x in pred:
    if x == 1.0:
        list.append(x)
print("The number of times 1 appears:")
len(list)

Target [ 1.  1.] -----> Predicted [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 

391

## MLPC (normalised data)

In [27]:
model = MLPC([20, 20], activation='relu', solver='adam', learning_rate_init=0.01, max_iter=100, shuffle=False, tol=1e-8, verbose=1)
model.fit(x_train, labels)
pred = [model.predict(np.array(cookie_dat).reshape(1, 3))[0] for cookie_dat in input_normalized]
print("Target {} -----> Predicted {}".format(labels[0:2], pred))
list = []
for x in pred:
    if x == 1.0:
        list.append(x)
print("The number of times 1 appears:")
len(list)

Iteration 1, loss = 7.53766062
Iteration 2, loss = 7.52876068
Iteration 3, loss = 7.52396838
Iteration 4, loss = 5.66196410
Iteration 5, loss = 5.66142930
Iteration 6, loss = 5.66084258
Iteration 7, loss = 5.65980503
Iteration 8, loss = 5.65866898
Iteration 9, loss = 5.65775078
Iteration 10, loss = 5.65721154
Iteration 11, loss = 7.51760392
Iteration 12, loss = 7.51731163
Iteration 13, loss = 7.51676869
Training loss did not improve more than tol=0.000000 for two consecutive epochs. Stopping.
Target [ 1.  1.] -----> Predicted [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

147

## MLPC (standardized data)

In [28]:
model = MLPC([20, 20], activation='relu', solver='adam', learning_rate_init=0.01, max_iter=200, shuffle=True, tol=1e-8, verbose=1)
model.fit(x_train, labels)
pred = [model.predict(np.array(cookie_dat).reshape(1, 3))[0] for cookie_dat in input_scaled]
print("Target {} -----> Predicted {}".format(labels[0:2], pred))
list = []
for x in pred:
    if x == 1.0:
        list.append(x)
print("The number of times 1 appears:")
len(list)

Iteration 1, loss = 7.55029771
Iteration 2, loss = 7.53838946
Iteration 3, loss = 7.52958731
Iteration 4, loss = 5.66348874
Iteration 5, loss = 5.66167357
Iteration 6, loss = 5.66149081
Iteration 7, loss = 5.66164199
Iteration 8, loss = 5.66159052
Iteration 9, loss = 5.66128781
Iteration 10, loss = 5.66077761
Iteration 11, loss = 7.52092679
Iteration 12, loss = 7.52056230
Iteration 13, loss = 7.52026597
Training loss did not improve more than tol=0.000000 for two consecutive epochs. Stopping.
Target [ 1.  1.] -----> Predicted [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

167

## MLPC min max

In [29]:
model = MLPC([20, 20], activation='relu', solver='adam', learning_rate_init=0.01, max_iter=200, shuffle=True, tol=1e-8, verbose=1)
model.fit(x_train_min, labels)
pred = [model.predict(np.array(cookie_dat).reshape(1, 3))[0] for cookie_dat in input_min_max]
print("Target {} -----> Predicted {}".format(labels[0:2], pred))
list = []
for x in pred:
    if x == 1.0:
        list.append(x)
print("The number of times 1 appears:")
len(list)

Iteration 1, loss = 0.76521554
Iteration 2, loss = 0.73560279
Iteration 3, loss = 0.71462227
Iteration 4, loss = 0.69917287
Iteration 5, loss = 0.68903651
Iteration 6, loss = 0.68430148
Iteration 7, loss = 0.68353751
Iteration 8, loss = 0.68548033
Iteration 9, loss = 0.68773784
Iteration 10, loss = 0.68882876
Training loss did not improve more than tol=0.000000 for two consecutive epochs. Stopping.
Target [ 1.  1.] -----> Predicted [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0

530